In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pip
!pip install 'keras<3.0.0' mediapipe-model-maker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
from google.colab import files

from mediapipe_model_maker import object_detector

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
train_dataset_path = "/content/drive/MyDrive/Detection-nuts-bolts.v1i.voc/train"
validation_dataset_path = "/content/drive/MyDrive/Detection-nuts-bolts.v1i.voc/valid"

In [ ]:
train_data = object_detector.Dataset.from_pascal_voc_folder(train_dataset_path)

val_data = object_detector.Dataset.from_pascal_voc_folder(validation_dataset_path)

In [ ]:
hparams = object_detector.HParams(batch_size=8, learning_rate=0.3, epochs=100, export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=object_detector.SupportedModels.MOBILENET_V2,
    hparams=hparams
)
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=val_data,
    options=options)

Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_1 (MobileNet)    {'2': (None, 64, 64, 24   2257984   
                             ),                                  
                              '3': (None, 32, 32, 32             
                             ),                                  
                              '4': (None, 16, 16, 96             
                             ),                                  
                              '5': (None, 8, 8, 320)             
                             , '6': (None, 8, 8, 128             
                             0)}                                 
                                                                 
 fpn_1 (FPN)                 {'5': (None, 8, 8, 128)   149056    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


30/30 [==============================] - 66s 441ms/step - total_loss: 14.3356 - cls_loss: 13.9902 - box_loss: 0.0058 - model_loss: 14.2806 - val_total_loss: 1.0421 - val_cls_loss: 0.8848 - val_box_loss: 0.0020 - val_model_loss: 0.9872
Epoch 2/100
30/30 [==============================] - 5s 176ms/step - total_loss: 1.1946 - cls_loss: 0.9375 - box_loss: 0.0040 - model_loss: 1.1396 - val_total_loss: 0.8976 - val_cls_loss: 0.7552 - val_box_loss: 0.0017 - val_model_loss: 0.8426
Epoch 3/100
30/30 [==============================] - 5s 175ms/step - total_loss: 0.9612 - cls_loss: 0.7566 - box_loss: 0.0030 - model_loss: 0.9062 - val_total_loss: 0.6888 - val_cls_loss: 0.5560 - val_box_loss: 0.0016 - val_model_loss: 0.6338
Epoch 4/100
30/30 [==============================] - 6s 207ms/step - total_loss: 0.7521 - cls_loss: 0.5655 - box_loss: 0.0026 - model_loss: 0.6972 - val_total_loss: 0.5471 - val_cls_loss: 0.4144 - val_box_loss: 0.0016 - val_model_loss: 0.4921
Epoch 5/100
30/30 [=================

In [ ]:
loss, data_metrics = model.evaluate(val_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {data_metrics}")

6/6 [==============================] - 1s 82ms/step - total_loss: 0.4720 - cls_loss: 0.3503 - box_loss: 0.0013 - model_loss: 0.4173


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.15s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.712
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.940
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.898
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.057
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.800
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.778
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.331
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.762
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.762
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.200
 Av

In [ ]:
model.export_model('bolt-detection-mp.tflite')
!ls exported_model
files.download('exported_model/bolt-detection-mp.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


bolt-detection-mp.tflite  float_ckpt.data-00000-of-00001  float_ckpt.index  metadata.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from tensorflow_model_optimization.python.core.quantization.keras import quantize


import tensorflow as tf

# class CustomLayer(tf.keras.layers.Layer):
#     def __init__(self, **kwargs):
#         super(CustomLayer, self).__init__(**kwargs)

#     def build(self, input_shape):
#         # Create a trainable weight variable for this layer.
#         self.kernel = self.add_weight("kernel", shape=[1],
#                                       initializer='uniform',
#                                       trainable=True)

#     def call(self, inputs):
#         return inputs * self.kernel

#     def get_config(self):
#         # Override `get_config` to enable model saving and loading.
#         base_config = super(CustomLayer, self).get_config()
#         return {**base_config}


# # Build a simple model including the custom layer
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(64, activation='relu', input_shape=(32,)),
#     CustomLayer(),
#     tf.keras.layers.Dense(10, activation='softmax')
# ])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# import numpy as np

# # Generate some random data to train on
# data = np.random.random((1000, 32))
# labels = np.random.random((1000, 10))

# # Train the model
# model.fit(data, labels, epochs=10, batch_size=32)
# # Save the model
# model.save('path_to_my_model.h5')

# # # Load the model
# custom_objects = {'CustomLayer': CustomLayer}
# model = tf.keras.models.load_model('path_to_my_model.h5', custom_objects=custom_objects)


# # Assuming 'CustomLayer' is a layer you have defined
# with quantize.quantize_scope({'CustomLayer': CustomLayer}):
#     # Your code to apply quantization here
#     model = quantize.quantize_apply(model)

qat_hparams = object_detector.QATHParams(learning_rate=0.3, batch_size=4, epochs=10, decay_steps=6, decay_rate=0.96)
model.quantization_aware_training(train_data, val_data, qat_hparams=qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(val_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


ValueError: Unable to clone model. This generally happens if you used custom Keras layers or objects in your model. Please specify them via `quantize_scope` for your calls to `quantize_model` and `quantize_apply`. [Layer <tf_keras.src.layers.core.activation.Activation object at 0x78755b5a9ea0> supplied to wrapper is not a supported layer type. Please ensure wrapped layer is a valid Keras layer.].

In [ ]:
new_qat_hparams = object_detector.QATHParams(learning_rate=0.9, batch_size=4, epochs=15, decay_steps=5, decay_rate=0.96)
model.restore_float_ckpt()
model.quantization_aware_training(train_data, val_data, qat_hparams=new_qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(val_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

Using existing files at /tmp/model_maker/object_detector/mobilenetv2_i256
Model: "retina_net_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net_3 (MobileNet)    {'2': (None, 64, 64, 24   2257984   
                             ),                                  
                              '3': (None, 32, 32, 32             
                             ),                                  
                              '4': (None, 16, 16, 96             
                             ),                                  
                              '5': (None, 8, 8, 320)             
                             , '6': (None, 8, 8, 128             
                             0)}                                 
                                                                 
 fpn_3 (FPN)                 {'5': (None, 8, 8, 128)   149056    
                             , '4': (Non

ValueError: Unable to clone model. This generally happens if you used custom Keras layers or objects in your model. Please specify them via `quantize_scope` for your calls to `quantize_model` and `quantize_apply`. [Layer <tf_keras.src.layers.core.activation.Activation object at 0x7875a1718640> supplied to wrapper is not a supported layer type. Please ensure wrapped layer is a valid Keras layer.].

In [ ]:
model.export_model('model_int8_qat.tflite')
!ls -lh exported_model
files.download('exported_model/model_int8_qat.tflite')